In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

In [30]:
class aslModel(nn.Module):
    def __init__(self):
        super(aslModel, self).__init__()
        # (w - k + 2p)/s + 1
        self.conv1 = nn.Conv2d(3, 32, 6, 1) # (200 - 6)/1 + 1 = 195
        self.conv2 = nn.Conv2d(32, 64, 6, 1) # (195 - 6)/1 + 1 = 190

        self.pool = nn.MaxPool2d(2, 2) # (190)/2 = 95
        self.drop1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(64, 128, 6, 1) # (95 - 6)/1 + 1 = 90
        self.conv4 = nn.Conv2d(128, 256, 5, 1) # (90 - 5)/1 + 1 = 86

        self.drop2 = nn.Dropout(0.25)
        # 43

        self.conv5 = nn.Conv2d(256, 256, 11, 1) # (43 - 11)/1 + 1 = 33
        self.conv6 = nn.Conv2d(256, 256, 14, 1) # (33 - 14)/1 + 1 = 20
        # 20
        
        self.drop3 = nn.Dropout(0.25)
        
        self.drop4 = nn.Dropout(0.5)
        self.drop5 = nn.Dropout(0.5)

        self.full1 = nn.Linear(256 * 10 * 10, 128)
        self.full2 = nn.Linear(128, 128)
        self.full3 = nn.Linear(128, 36)
    
    def forward(self, x):
        x = self.drop1(self.pool(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.drop2(self.pool(F.relu(self.conv4(F.relu(self.conv3(x))))))
        x = self.drop3(self.pool(F.relu(self.conv6(F.relu(self.conv5(x))))))

        x = x.view(-1, 256 * 10 * 10)
        x = self.drop4(F.relu(self.full1(x)))
        x = self.drop5(F.relu(self.full2(x)))
        x = self.full3(x)
        return x

In [3]:
import torch
import torchvision
from torchvision import transforms, datasets

In [26]:
num_epochs = 1
batch = 10
learning_rate = 0.01

In [27]:
data_transform = transforms.Compose([
    transforms.Resize((200,200)),
    transforms.ToTensor()
])
massey_dataset = datasets.ImageFolder(root='massey',
                                           transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(massey_dataset,
                                             batch_size=batch, shuffle=True)

In [31]:
model = aslModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()

loss = []
accuracy = []

In [32]:
# Train the model
for epoch in range(num_epochs):
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(dataset_loader):
        if(Y_train_batch.shape[0]<batch):
            continue
            
        output = model(X_train_batch)
        curr_loss = criterion(output, Y_train_batch)
        loss.append(curr_loss.item())

        optimizer.zero_grad()
        curr_loss.backward()
        optimizer.step()

        _, predicted = torch.max(output.data, 1)
        correct = (predicted == Y_train_batch).sum().item()
        accuracy.append(correct/Y_train_batch.size(0))
        print(predicted)
        print(Y_train_batch)
        print(accuracy[-1])

tensor([ 7, 28, 15,  7, 21,  7,  7,  7, 28,  7])
tensor([35, 22, 35,  4,  6,  0,  2, 28, 19, 35])
0.0
tensor([26, 26,  0,  0, 19,  0, 28,  0,  0,  0])
tensor([16,  0,  9,  6, 17,  3, 16,  9, 30, 10])
0.0
tensor([28, 32,  9, 21, 28,  6,  7, 28, 34, 15])
tensor([ 0, 11, 19, 15,  3, 31, 27, 14, 32, 18])
0.0
tensor([ 9,  9, 27, 28,  9, 27,  2,  9, 20,  9])
tensor([20,  8, 16, 17,  9, 21,  1,  7, 22, 12])
0.1
tensor([16,  9, 16, 19, 33, 16, 34,  3, 22,  4])
tensor([13,  7, 35, 14, 16, 15, 28,  1, 12,  0])
0.0
tensor([35, 35, 11,  6, 15,  9, 27,  2, 35, 22])
tensor([33, 15, 31, 28, 30, 28, 14, 17, 10,  9])
0.0
tensor([35, 35, 35, 35, 35, 35, 35, 35, 35, 35])
tensor([29, 31,  6, 35,  1,  9,  2,  3, 26, 29])
0.1
tensor([35, 35, 35, 35, 35, 35, 35, 35, 35, 35])
tensor([ 8, 18, 17, 11, 28, 30, 28, 22,  0, 15])
0.0
tensor([14,  9, 35, 13, 16,  9, 16, 17, 16, 35])
tensor([10,  5, 12, 14, 10,  8,  9, 32, 35,  4])
0.0
tensor([35, 35, 35, 35, 35, 35, 35, 35, 35, 35])
tensor([ 0,  7, 24, 17,  4, 32, 1

KeyboardInterrupt: 